# <b> Only problem 1 and 6 will be graded. </b>

## Problem 1 : Integer program


Solve the following program by using linprog function and branch and bound method:
$$Objective : max(3x + 4y) $$
$$\begin{align*}
  x + 2y &\leq 7 \\
  3x  - y &\geq 0 \\
  x -  y &\leq  2 \\
  x, y  &\in Z^+ \cup \{0\} \\
\end{align*}$$

    Solution:

scipy.optimize.linprog: **minimize** a linear objective function subject to linear equality and inequality constraints.

$$Objective : min(-3x-4y) $$
$$\begin{align*}
  x + 2y & \leq 7 \\
  -3x + y & \leq 0 \\
  x -  y & \leq  2 \\
  x, y &\geq 0 \\
\end{align*}$$

In [1]:
import numpy as np
from scipy.optimize import linprog

c = [-3, -4]

A = [[1, 2],
     [-3, 1],
     [1, -1]]

b = [7, 0, 2]

x_bounds = (0, None)
y_bounds = (0, None)

result = linprog(c, A_ub=A, b_ub=b, bounds=[x_bounds, y_bounds], method='highs')

print(f"Optimal value: {-result.fun}, x: {result.x[0]}, y: {result.x[1]}")

Optimal value: 17.666666666666668, x: 3.6666666666666665, y: 1.6666666666666667


In [2]:
# Subproblem 1: x <= 3

x_bounds_sub1 = (0, 3)

result_sub1 = linprog(c, A_ub=A, b_ub=b, bounds=[x_bounds_sub1, y_bounds], method='highs')

print(f"Subproblem 1 (x <= 3): Optimal value: {-result_sub1.fun}, x: {result_sub1.x[0]}, y: {result_sub1.x[1]}")

Subproblem 1 (x <= 3): Optimal value: 17.0, x: 3.0, y: 2.0


In [3]:
# Subproblem 2: x >= 4

x_bounds_sub2 = (4, None)

result_sub2 = linprog(c, A_ub=A, b_ub=b, bounds=[x_bounds_sub2, y_bounds], method='highs')

if result_sub2.success:
    print(f"Subproblem 2 (x >= 4): Optimal value: {-result_sub2.fun}, x: {result_sub2.x[0]}, y: {result_sub2.x[1]}")
else:
    print("Subproblem 2 (x >= 4): No feasible solution.")

Subproblem 2 (x >= 4): No feasible solution.


**Conclusion**:

Since Subproblem 1 provides a feasible integer solution\
and Subproblem 2 has no feasible solution,\
the optimal integer solution to the original problem is:

$\therefore \text{Optimal value}= 17,\quad x= 3,\quad y= 2$

# Problem 6 : Instraham


After several crises, Hamtaro is fed up with the manufacturing business and is now looking for new business opportunities. He finds out that opening social media platform could make a hefty sum of money. Moreover, since hamsters do not currently have a social media site, Hamtaro can monopolize the market easily. Therefore, he creates Instraham, the first social media website of hamsters, by hamsters, for hamsters.

After consulting with Koushi-kun, Hamtaro figures out that social network platforms often have the features shown in the table below. For each feature, the CPU load and storage load are shown with its associated business value score.

|Feature name| CPU load (%) | storage load (%) | business value score |
|:-:|:-:|:-:|:-:|
| A | 20 | 30| 10|
| B | 10 | 5| 5|
| C| 30 | 10| 10|
| D | 5 | 10| 3|
| F | 15 | 30| 10|
| G | 60 | 70| 30|
| H | 80 | 80| 80|
| I | 10 | 50| 20|
| J | 3 | 50| 5|

Feature A, and J is mandatory while the rest is optional. The objective is to maximize the business value score of the website while not overloading CPU and storage servers. His engineering friend, Taisho-kun, also suggests him that he could improve the website efficiency by performing the following operations:

- Feature compression. This method will reduce both CPU and storage load by half, but it also reduces the business value to 55% of the original value. Every feature could be compressed, but the number of compressed features in the website is limited to two.

-  The usage of storage efficient algorithm. By using this method, the feature storage load is reduced by half but it also doubles the CPU load. However, only feature H, I, J can use this method. This method could not be used concurrently with feature compression.

From this information, which features should Hamtaro develop? ~~Use Amdahl's law to find the best speedup~~. Formulate the problem as an integer program and solve for an optimal solution.

<b> Note : This problem is based on the blog (https://engineering.fb.com/2021/07/29/data-infrastructure/linear-programming/) </b>


    Solution:

**Decision variables**:

- $x_i$: Binary variable (0 or 1) for whether feature  i  is *selected* for the platform.
- $c_i$: Binary variable (0 or 1) for whether feature  i  is *compressed*.
- $s_i$: Binary variable (0 or 1) for whether feature  i  is uses the *storage-efficient* algorithm.

**Objective**:\
maximize the business value score
$$max(\ \sum_{i}(x_i v_i - 0.45 c_i v_i)\ )$$
; $v_i$ is business value score of feature i

**Constraints**:

- Total CPU Load
$$
\sum Normal - Feature\ compression + Storage\ efficient\ algorithm
\leq 100
$$
$$
\sum_{i} (x_i \cdot CPU_i)
- (c_i \cdot x_i \cdot 0.5 CPU_i)
+ (s_i \cdot x_i \cdot 2 CPU_i)
\leq 100
$$
$$
\therefore
\sum_{i} (x_i \cdot CPU_i)(1-0.5c_i+2s_i)
\leq 100
$$
; $CPU_i$ is CPU Load of feature i

- Total Storage Load
$$
\sum_{i} (x_i \cdot Storage_i)
- (c_i \cdot x_i \cdot 0.5Storage_i)
- (s_i \cdot x_i \cdot 0.5Storage_i)
\leq 100
$$
$$
\therefore
\sum_{i} (x_i \cdot Storage_i)(1-0.5c_i-0.5s_i)
\leq 100
$$
; $Storage_i$ is storage load of feature i

- Mandatory features
$$x_A=1,\quad x_J=1$$

- If use Feature compression must be selected first
$$c_i \leq x_i$$

- If use Storage efficient algorithm must be selected first
$$s_i \leq x_i$$

- Feature compression limit
$$\sum_i c_i \leq 2$$

- Storage efficient algorithm can use only by H, I, J
$$s_A=0,\ s_B=0,\ s_C=0,\ s_D=0,\ s_F=0,\ s_G=0$$

- Storage efficient algorithm could not be used concurrently with feature compression
$$s_i + c_i \leq 1$$

In [4]:
import pulp

prob = pulp.LpProblem("Instraham", pulp.LpMaximize)

# Data
features = ['A', 'B', 'C', 'D', 'F', 'G', 'H', 'I', 'J']
cpu = {'A': 20, 'B': 10, 'C': 30, 'D': 5, 'F': 15, 'G': 60, 'H': 80, 'I': 10, 'J': 3}
storage = {'A': 30, 'B': 5, 'C': 10, 'D': 10, 'F': 30, 'G': 70, 'H': 80, 'I': 50, 'J': 50}
business_value = {'A': 10, 'B': 5, 'C': 10, 'D': 3, 'F': 10, 'G': 30, 'H': 80, 'I': 20, 'J': 5}

# Decision variables
x = pulp.LpVariable.dicts("select", features, cat='Binary')  # Feature selection
c = pulp.LpVariable.dicts("compress", features, cat='Binary')  # Compression
s = pulp.LpVariable.dicts("storage_efficient", features, cat='Binary')  # Storage-efficient

# Objective function: Maximize business value
prob += pulp.lpSum([business_value[f] * (x[f] - 0.45 * c[f]) for f in features])

In [5]:
# Constraints

# CPU Load
prob += pulp.lpSum([cpu[f]*x[f] - 0.5*cpu[f]*c[f] + cpu[f]*s[f] for f in features]) <= 100

# Storage Load
prob += pulp.lpSum([storage[f]*x[f] - 0.5*storage[f]*c[f] - 0.5*storage[f]*s[f] for f in features]) <= 100

# Mandatory features: A and J
prob += x['A'] == 1
prob += x['J'] == 1

# Feature Compression
prob += pulp.lpSum(c[f] for f in features) <= 2  # Limit
for f in features:
    prob += c[f] <= x[f] # must be selected features

# Storage-efficient algorithm
# Only H, I, J can use storage-efficient
for f in ['A', 'B', 'C', 'D', 'F', 'G']:
    prob += s[f] == 0
for f in features:
    prob += s[f] <= x[f] # must be selected features
    prob += s[f] + c[f] <= 1  # no concurrent

# Solve the problem
status = prob.solve()

# Print results
print("\nOptimal Feature Selection:")
print("-" * 50)
print("Selected Features:")
for f in features:
    if x[f].value() > 0.5:  # Using 0.5 to handle floating-point imprecision
        optimizations = []
        if c[f].value() > 0.5:
            optimizations.append("compressed")
        if s[f].value() > 0.5:
            optimizations.append("storage-efficient")
        opt_str = f" ({', '.join(optimizations)})" if optimizations else ""
        print(f"- Feature {f}{opt_str}")

print("\nTotal Business Value:", pulp.value(prob.objective))
print("CPU Load:", sum(cpu[f] * x[f].value() - 0.5 * cpu[f] * c[f].value() + cpu[f] * s[f].value() for f in features))
print("Storage Load:", sum(storage[f] * x[f].value() - 0.5 * storage[f] * c[f].value() - 0.5 * storage[f] * s[f].value() for f in features))

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/pupipatsingkhorn/miniconda3/envs/datascience/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/m6/fz_qjnl51s70hy69d_st2z240000gn/T/45aa897acc4d41018b92dfb0b39974f9-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/m6/fz_qjnl51s70hy69d_st2z240000gn/T/45aa897acc4d41018b92dfb0b39974f9-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 43 COLUMNS
At line 241 RHS
At line 280 BOUNDS
At line 308 ENDATA
Problem MODEL has 38 rows, 27 columns and 125 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 82.95 - 0.00 seconds
Cgl0002I 6 variables fixed
Cgl0003I 1 fixed, 0 tightened bounds, 1 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 1 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 1 strengthene

**Conclusion**:

Optimal Feature Selection:

Selected Features:
- Feature A (compressed)
- Feature B
- Feature C
- Feature H (compressed)
- Feature J (storage-efficient)

Total Business Value: 69.5\
CPU Load: 96.0\
Storage Load: 95.0